## Base Models

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [22]:
AKF_final_test = pd.read_csv('./AKF_final_test.csv.gz', compression='gzip')
AKF_final_test_KNN = pd.read_csv('./AKF_KNN_final_test.csv.gz', compression='gzip')


In [42]:
hadm_features = AKF_final_test.loc[:, '(\'min\', 50861)' : '(\'above_max\', 51514)']
hadm_target = AKF_final_test.loc[:, 'AKF']

In [43]:
# # Creatinine - 50912
# hadm_features = hadm_features.loc[:, ['(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]

# # Least Missingness
# # [51221, 51265, 51222, 51301, 51279, 51277, 51248, 51249, 51250]
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)']]

# Least Missingness + Creatinine
hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
                                        '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
                                        '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
                                        '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
                                        '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
                                        '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
                                        '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
                                        '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
                                        '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)',
                                     '(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]


In [44]:
X_train, X_test, y_train, y_test = train_test_split(hadm_features, hadm_target, stratify=hadm_target, random_state=0)


In [6]:
# # # Optional cell for NearMiss
# # from imblearn.under_sampling import NearMiss
# # from imblearn.over_sampling import SMOTE
# # sm = SMOTE()
# # nm = NearMiss()
# # X, y = nm.transform(hadm_features, hadm_target)
# # hadm_features, hadm_target = X, y

# # Simulate NearMiss
# train_data = pd.concat([X_train, y_train], axis=1)
# test_data = pd.concat([X_test, y_test], axis=1)

# AKF = train_data[train_data.AKF==1]
# non_AKF = train_data[train_data.AKF==0]

# non_AKF = non_AKF.sample(n=len(AKF))

# undersampled_train_data = pd.concat([AKF, non_AKF], axis=0)
# undersampled_train_data = undersampled_train_data.sample(frac=1)

# X_train = undersampled_train_data.drop('AKF', axis=1)
# y_train = undersampled_train_data['AKF']

# X_test = test_data.drop('AKF', axis=1)
# y_test = test_data['AKF']


## Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [49]:
%%time

# # L2
# params_log = [ {'C':np.logspace(-4, 3, 20)} ]
# logReg = LogisticRegression(max_iter = 2000, class_weight = "balanced")

# L1
params_log = [ {'C':np.logspace(-4, -1, 20)} ]
logReg = LogisticRegression(penalty = 'l1', max_iter = 2000, class_weight = "balanced", solver = 'liblinear')

grid_log = GridSearchCV(estimator = logReg, param_grid = params_log, cv = 5)
grid_log.fit(X_train, y_train)
grid_log.best_score_

CPU times: user 14.3 s, sys: 2.96 s, total: 17.3 s
Wall time: 3.32 s


0.8987674609695974

In [50]:
grid_log.best_params_

{'C': 0.0006158482110660267}

In [51]:
list(zip(X_train.columns, grid_log.best_estimator_.coef_[0]))


[("('max', 51221)", 0.0),
 ("('above_max', 51221)", 0.0),
 ("('mean', 51221)", 0.0),
 ("('abn_percent', 51221)", 0.0),
 ("('max', 51265)", 0.0),
 ("('above_max', 51265)", 0.0),
 ("('mean', 51265)", 0.0),
 ("('abn_percent', 51265)", 0.0),
 ("('max', 51222)", 0.0),
 ("('above_max', 51222)", 0.0),
 ("('mean', 51222)", 0.0),
 ("('abn_percent', 51222)", 0.0),
 ("('max', 51301)", 0.0),
 ("('above_max', 51301)", 0.0),
 ("('mean', 51301)", 0.0),
 ("('abn_percent', 51301)", 0.0),
 ("('max', 51279)", 0.0),
 ("('above_max', 51279)", 0.0),
 ("('mean', 51279)", 0.0),
 ("('abn_percent', 51279)", 0.0),
 ("('max', 51277)", 0.0),
 ("('above_max', 51277)", 0.0),
 ("('mean', 51277)", 0.0),
 ("('abn_percent', 51277)", 0.0),
 ("('max', 51248)", 0.0),
 ("('above_max', 51248)", 0.0),
 ("('mean', 51248)", 0.0),
 ("('abn_percent', 51248)", 0.0),
 ("('max', 51249)", 0.0),
 ("('above_max', 51249)", 0.0),
 ("('mean', 51249)", 0.0),
 ("('abn_percent', 51249)", 0.0),
 ("('max', 51250)", 0.0),
 ("('above_max', 51250

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [52]:
pred = grid_log.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

Accuracy Score: 87.78%
_______________________________________________
Classification Report:	Precision Score: 54.55%
			Recall Score: 77.89%
			F1 score: 64.16%
_______________________________________________
Confusion Matrix: 
 [[1559  185]
 [  63  222]]



## Random Forest

In [13]:
from sklearn import ensemble
import sklearn.model_selection as ms

randomForest = ensemble.RandomForestClassifier(class_weight = "balanced")

In [14]:
%%time

grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 20),
    'n_estimators': range(10, 110, 10)
}

grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

CPU times: user 7.98 s, sys: 764 ms, total: 8.74 s
Wall time: 44.9 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 20),
                         'n_estimators': range(10, 110, 10)},
             scoring='accuracy')

In [15]:
# Get most important features
tree_final = grid_search_forest.best_estimator_
feature_importance = tree_final.feature_importances_
feature_importance = list(zip(X_train.columns, feature_importance))

feature_importance.sort(key = lambda x: x[1], reverse = True)
feature_importance

[("('max', 50912)", 0.3945927536480366),
 ("('above_max', 50912)", 0.3942217221682857),
 ("('abn_percent', 50912)", 0.15528583495965484),
 ("('mean', 50912)", 0.05589968922402293)]

In [16]:
grid_search_forest.best_params_

{'criterion': 'gini', 'max_depth': 3, 'n_estimators': 30}

In [17]:
grid_search_forest.best_score_

0.9059016393442623

In [18]:
print("The training error is: %.5f" % (grid_search_forest.score(X_train, y_train)))
print("The test     error is: %.5f" % (grid_search_forest.score(X_test, y_test)))

The training error is: 0.90689
The test     error is: 0.90855


In [19]:
rf_predictions = tree_final.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, rf_predictions)}\n")

Accuracy Score: 90.86%
_______________________________________________
Classification Report:	Precision Score: 63.10%
			Recall Score: 83.10%
			F1 score: 71.73%
_______________________________________________
Confusion Matrix: 
 [[806  69]
 [ 24 118]]

